<a href="https://colab.research.google.com/github/cr2uchile/Quality_Control_FireScarCL/blob/main/3_fixes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import glob
import pandas as pd

from osgeo import gdal
# OBS printGetHistogram(0,2,2, approx_ok=0 ) !! Without change gives wrong values for large rasters

!pip install pyshp
import shapefile
# FIX fireRegion 
# fireID   shouldbe   is
#ID183621  CL-BI  CL-LL
#ID192587  CL-BI  CL-LL
#ID193098  CL-BI  CL-LL



     |████████████████████████████████| 219 kB 5.4 MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37325 sha256=a4ce396c89d7f4f3b1758e85fb2c83edb5d6e7e19dba0d3b4e59eb8dc39875f0
  Stored in directory: /root/.cache/pip/wheels/43/f8/87/53c8cd41545ba20e536ea29a8fcb5431b5f477ca50d5dffbbe
Successfully built pyshp


In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 
#listOfFiles = os.listdir('drive/MyDrive')

Mounted at /content/drive/


In [ ]:
sys.path.insert(0,'/content/drive/Shareddrives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/ColabNotebooks/')
from common import *

print (output_path)
print (conaf_filename)

drive/Shared drives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Report/
RegistroIncendiosCONAF_depurada_Final.csv


In [ ]:
# READ  location output
# put needed things in location output.. (Name, IgnDate, CtrlDate, observations, )
tag='20211113_V5'
file_name=output_path+'output_location_'+tag+'.xlsx'
dflocations = pd.read_excel(file_name)
# SET INDEX
dflocations.set_index('fireId', inplace=True)
print(dflocations.head)

# READ  supermaster for fixing dbfs
tag='20211115_V8'
file_name=output_path+'output_SUPERMASTER_'+tag+'.xlsx'
dfsm = pd.read_excel(file_name)
dfsm = dfsm.loc[dfsm['FireScar'] == 1]
dfsm.rename(columns={'FireID':'fireId'}, inplace=True)
dfsm.set_index('fireId', inplace=True)
dfsm.sort_index(inplace=True)
print(dfsm.head)
print('len dfsm', len(dfsm))
print(dfsm.columns)


<bound method NDFrame.head of          Region_conaf  ...                       observationRegions
fireId                 ...                                         
ID100000        Maule  ...                                      NaN
ID10002        BioBio  ...  Nuble corresponds to the ignition point
ID100027        Maule  ...                                      NaN
ID100032        Maule  ...                                      NaN
ID100041        Maule  ...                                      NaN
...               ...  ...                                      ...
ID99979         Maule  ...                                      NaN
ID9998         BioBio  ...  Nuble corresponds to the ignition point
ID99985         Maule  ...                                      NaN
ID99995         Maule  ...                                      NaN
ID99998         Maule  ...                                      NaN

[8153 rows x 28 columns]>
<bound method NDFrame.head of           FireSeason  ...    

In [ ]:
print(dfsm['ControlDate_CONAF']['ID8301'])
mylat = dfsm['Latitude [°]']['ID8301']
print(mylat)
print(isinstance(mylat, float))
dfsm.fillna('', inplace=True)
print(dfsm.head)

1985-12-26
-39.2612
True
<bound method NDFrame.head of           FireSeason  ...                                       Observations
fireId                ...                                                   
ID100000        2003  ...                                                   
ID10002         1986  ...  The ignition point is located in the current C...
ID100027        2003  ...                                                   
ID100032        2003  ...                                                   
ID100041        2003  ...                                                   
...              ...  ...                                                ...
ID99979         2003  ...                                                   
ID9998          1986  ...  The ignition point is located in the current C...
ID99985         2003  ...                                                   
ID99995         2003  ...                                                   
ID99998         2003 

In [ ]:
def getPathFor(fireId):
  return base_path+dflocations.loc[fireId]['folderRegion']+'/'+dflocations.loc[fireId]['folderSeason']+'/'
  #['folderRegion'].values[fireId]
  #return dflocations[fireId]['folderRegion']+'/'+dflocations[fireId]['folderSeason']

# Ref FireScar_CL-BI_ID151372_u390_20101211.tif
def getSRTifNames(firescarPath):
  fileName =os.path.basename(firescarPath)
  dirName = os.path.dirname(firescarPath)
  print(fileName)
  (fileNameOnly, fileExt)=os.path.splitext(fileName) 
  (fireType, fireRegion, fireId, fireThreshold, fireDate)=fileNameOnly.split("_")
  sname ='Severity'+'_'+ fireRegion+'_'+ fireId+'_'+ fireThreshold+'_'+ fireDate+'.tif'
  rname ='RdNBR'+'_'+ fireRegion+'_'+ fireId+'_'+ fireThreshold+'_'+ fireDate+'.tif'
  fname = fileName
  return [fname, sname, rname]

print(getPathFor('ID103632' ))

drive/Shared drives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Fire_CL-AR_Araucania/FireScar_CL-AR_Araucania_2003/


In [ ]:
# DONE [left]:  (3, 2) .. 3 files
def rename_fireRegion(fireId, currentFireRegion, newFireRegion):
  # need full in path, out path
  listFiles = filter(os.path.isfile, glob.glob(getPathFor(fireId) + '*'+currentFireRegion+'_'+fireId+'*') ) 
  for filePath in listFiles:  
    print(filePath) 
    print(filePath.replace(currentFireRegion, newFireRegion)) 
    os.rename(filePath,filePath.replace(currentFireRegion, newFireRegion))
    print('renamed '+currentFireRegion+' to '+newFireRegion)

# DONE [left]:  (551, 2) .. 551 files to rename
def rename_fireDate(fireId, currentFireDate, newFireDate):
  listFiles = filter(os.path.isfile, glob.glob(getPathFor(fireId) + '*'+fireId+'*'+currentFireDate+'*') ) 
  for filePath in listFiles:  
    print(filePath) 
    print(filePath.replace(currentFireDate, newFireDate)) 
    os.rename(filePath,filePath.replace(currentFireDate, newFireDate))
    print('renamed '+currentFireDate+' to '+newFireDate)

# MOVE/create FILES TO THE CORRECT SEASON !!!!
# unnecessary

def crop_replace(firepath, fname, shapebox2):
  originalTif = os.path.join(firepath, fname)
  # gets tifs bounding box, to see if it already has the desired size
  t1 = gdal.Open(originalTif)
  geoTransform = t1.GetGeoTransform()
  upperleftx = geoTransform[0]
  upperlefty = geoTransform[3]
  lowerrightx = upperleftx + geoTransform[1] * t1.RasterXSize
  lowerrighty = upperlefty + geoTransform[5] * t1.RasterYSize
  tifbox =  [upperleftx, upperlefty, lowerrightx, lowerrighty]
  t1 = None
  print('tifbox:')
  print(tifbox)
  print('shapebox2:')
  print(shapebox2)
  if (shapebox2 == tifbox):
    print(" Already cropped "+originalTif)
  else:
    # outputType=gdal.GDT_Byte,
    croppedTif = os.path.join(firepath, fname+'NEWCROP.tif')
    ds1 = gdal.Translate(croppedTif, originalTif, format='GTiff',  creationOptions= ['TFW=NO', 'COMPRESS=LZW'], projWin=shapebox2)
    ds1 = None   # to close and flush
    # MOVE old files to backup, rename new files to old.
    backup_dir = os.path.join(base_path,'Report','backup')
    print ('rename '+originalTif +' to '+os.path.join(backup_dir, fname))
    os.rename(originalTif, os.path.join(backup_dir, fname))
    print ('rename '+croppedTif +' to '+originalTif)
    os.rename(croppedTif, originalTif)


# left 8 fires .. 8x3 tif files to crop
def crop_tifFile(fireId):
  print (fireId)              
  # Get bounding Box from Shapefile
  firepath=getPathFor(fireId)
  listFiles = filter(os.path.isfile, glob.glob(firepath + 'FireScar*'+fireId+'*.shp') ) 
  for shapePath in listFiles:
    print (shapePath) 
    sf = shapefile.Reader(shapePath)
    print (sf.shapes)
    shapebox = sf.bbox
    # to get [ulx, uly, lrx, lry]  upper left, lower right geometry
    shapebox2 = [shapebox[0], shapebox[3], shapebox[2], shapebox[1]]
    print(shapebox2)
    fileName =os.path.basename(shapePath)
    (fileNameOnly, fileExt)=os.path.splitext(fileName)
    firescarFullPath = os.path.join(firepath,fileNameOnly+'.tif')
    # Crop 3 tifs with bbox if geometry is different
    [fname, sname, rname] = getSRTifNames(firescarFullPath)
    # projWin --- subwindow in projected coordinates to extract: [ulx, uly, lrx, lry]  / upper left, lower right
    # to compress: creationOptions= ['TFW=YES', 'COMPRESS=LZW']
    crop_replace(firepath, fname, shapebox2)
    crop_replace(firepath, sname, shapebox2)
    crop_replace(firepath, rname, shapebox2)


In [ ]:
# DONE
rename_fireRegion('ID183621', 'CL-LL', 'CL-BI')
rename_fireRegion('ID192587', 'CL-LL', 'CL-BI')
rename_fireRegion('ID193098', 'CL-LL', 'CL-BI')
# DONE

In [ ]:
# DONE
# rename_fireDate('ID189023', '20161009', '20161109' )
# load fire dates differences
file_name=output_path+'fireDateDif.csv'
dfDatesDif = pd.read_csv(file_name, index_col=0, sep = "\s+", dtype=str)
#print(dfDatesDif)
i=1
for index, row in dfDatesDif.iterrows():
  print(i)
  #print (index, row['currentFireDate'], row['newFireDate'])
  rename_fireDate(index, row['currentFireDate'], row['newFireDate'] )
  i=i+1


In [ ]:
# DONE 
# CROP 8 x 3 TIFS with bounding box 
crop_tifFile('ID189054')
crop_tifFile('ID189174')
crop_tifFile('ID189255')
crop_tifFile('ID189260')
crop_tifFile('ID189272')
crop_tifFile('ID189274')
crop_tifFile('ID189277')

### TODOOOOOOOOOO
crop_tifFile('ID189275')


          

In [ ]:
# read / write a dbf / rename columns / reorder columns / replace data (dates, fireName) / add columns (ignition point)

In [ ]:
def create_folder(backup_region_path):
  print(backup_region_path)      
  if not os.path.exists(backup_region_path):
    os.makedirs(backup_region_path)
    print(backup_region_path+ " folder was created!")
    
def remove_xml_file(fileExt, filePath):
  if fileExt=='.xml':
    os.remove(filePath)
    print(filePath)
    print('removed')

In [ ]:
# needs: IgnitionDate, ControlDate, Name,  lat, lon, Overlap IDS, from dfsm (dataframe super master) 
def fix_firescar_dbf(fireId, inFilePath, outFilePath):
  #listFiles = filter(os.path.isfile, glob.glob(base_path +'Fire_CL-BI_BioBio/FireScar_CL-BI_BioBio_1985/'+'*.dbf') ) 
#for filePath in listFiles:  
#  print(filePath) 
#r = shapefile.Reader(os.path.join(output_path, 'testin', 'FireScar_CL-CO_ID20463_u700_19890120.dbf'))
#w = shapefile.Writer(os.path.join(output_path, 'testout', 'FireScar_CL-CO_ID20463_u700_19890120.dbf'), encoding="utf8")
  r = shapefile.Reader(inFilePath)
  w = shapefile.Writer(outFilePath)
  # EXTRA check field names consistency
  assert (r.fields[1][0]=='NIR_min'  and r.fields[4][0]=='AreaINC_ha')


  # RENAME, 40 to FireName, Character 10 to FireID
  w.fields= [['NIRMin', 'N', r.fields[1][2], r.fields[1][3]], ['TotalArea', 'N', r.fields[2][2], r.fields[2][3]], ['Season', 'N', 4, 0], ['AreaCONAF', 'N', r.fields[4][2], r.fields[4][3]], 
            ['AreaUnchS', 'N', r.fields[5][2], r.fields[5][3]], ['AreaLowS', 'N', r.fields[6][2], r.fields[6][3]], ['AreaHighS', 'N', r.fields[7][2], r.fields[7][3]], 
            ['CtrlDate', 'C', 10, 0], ['AreaModS', 'N', r.fields[9][2], r.fields[9][3]], ['IgnDate', 'C', 10, 0], ['Region', 'C', 14, 0], 
            ['NDVIMean', 'N', r.fields[12][2], r.fields[12][3]], ['FireName', 'C', len(dfsm['FireName_CONAF'][fireId])+1, 0], ['PolygArea', 'N', r.fields[14][2], r.fields[14][3]], 
             ['FireID', 'C', 10, 0]]
  #w.fields = r.fields[1:] # skip first deletion field 

  # ADD NEW FIELDS
  w.field('IgnLat', 'N', decimal=4)
  w.field('IgnLon', 'N', decimal=4)
  w.field('OverlapIDs', 'C', '40')

  # REORDER OK
  myorder = [14, 10, 2, 12, 9, 7, 15, 16, 11, 13, 0, 1, 4, 5, 8, 6, 3, 17]
  w.fields = [w.fields[i] for i in myorder]
  print(w.fields)
  # needs: IgnitionDate, ControlDate, Name,  lat, lon, Overlap IDS 
  for shaperec in r.iterShapeRecords():
    newline = shaperec.record  
    newline[7] =  dfsm['ControlDate_CONAF'][fireId]
    newline[9] =  dfsm['IgnitionDate_CONAF'][fireId]
    newline[12] = dfsm['FireName_CONAF'][fireId]
    newline[14] = 'ID'+str(newline[14])
    newline.append(dfsm['Latitude [°]'][fireId]) # OBS: IS FLOAT
    newline.append(dfsm['Longitude [°]'][fireId])
    newline.append(dfsm['OverlapIDs'][fireId])
    newline = [newline[i] for i in myorder]
    print (newline)
    w.record(*newline)
    w.shape(shaperec.shape)  
  w.close()
  # test
  #w2 = shapefile.Reader())
  #print (w2.fields)
  #print (w2.records)


In [ ]:
def check_firescar_dbf(fireId, filePath, backupFolder):
  print('check firescar dbf for ',filePath)
  print('with backup folder ',backupFolder)

  fileName=os.path.basename(filePath)
  dirName = os.path.dirname(filePath) 
  # first identify dbf, shp and shx and MOVE to backup with same structure (create if not existing)
  #print (fileName)
  (fileNameOnly, fileExt)=os.path.splitext(fileName) 
  f2 = os.path.join(dirName, fileNameOnly+'.shp')
  f3 = os.path.join(dirName, fileNameOnly+'.shx')
  assert (os.path.getsize(filePath) and os.path.getsize(f2) and os.path.getsize(f3))  
   #os.path.getsize(path) 
  print('OK ALL PATHS'+f3)
  # OPEN AND CHECK FIELDS!
  r = shapefile.Reader(filePath)
  print("R.FIELDS!")
  print(r.fields)
  ###[('DeletionFlag', 'C', 1, 0), ['NIR_min', 'N', 4, 0], ['AreaTotal', 'N', 17, 15], ['Temporada', 'N', 4, 0], ['AreaINC_ha', 'N', 4, 2], ['AreaUnchS', 'N', 1, 0], ['AreaLowS', 'N', 1, 0], 
  # ['AreaHighS', 'N', 18, 15], ['Control', 'C', 8, 0], ['AreaMedS', 'N', 17, 15], ['Inicio', 'C', 14, 0], ['Region', 'C', 8, 0], ['NDVI_mean', 'N', 19, 15], ['Nombre_Inc', 'C', 15, 0], 
  # ['Area_Poly', 'N', 18, 15], ['KEY', 'N', 5, 0]]

  #### All files have different definitions for the ranges .. only names are standard
  #if (r.fields==[('DeletionFlag', 'C', 1, 0),
  #      ['NIR_min', 'N', 3, 0], ['AreaTotal', 'N', 19, 15], ['Temporada', 'N', 4, 0], ['AreaINC_ha', 'N', 6, 4], ['AreaUnchS', 'N', 1, 0], 
  #              ['AreaLowS', 'N', 1, 0], ['AreaHighS', 'N', 19, 15], ['Control', 'C', 14, 0], ['AreaMedS', 'N', 1, 0], ['Inicio', 'C', 14, 0], ['Region', 'C', 8, 0], 
  #              ['NDVI_mean', 'N', 19, 15], ['Nombre_Inc', 'C', 11, 0], ['Area_Poly', 'N', 20, 15], ['KEY', 'N', 5, 0]]):

  if (r.fields[1][0]=='NIR_min'  and r.fields[4][0]=='AreaINC_ha'):
    # then send to fix function 
    # move original files to backup
    print ('MOVE '+filePath +' to '+os.path.join(backupFolder, fileName))
    os.rename(filePath, os.path.join(backupFolder, fileName))
    print ('MOVE '+f2 +' to '+os.path.join(backupFolder, fileNameOnly+'.shp'))
    os.rename(f2, os.path.join(backupFolder, fileNameOnly+'.shp'))
    print ('MOVE '+f3 +' to '+os.path.join(backupFolder, fileNameOnly+'.shx'))
    os.rename(f3, os.path.join(backupFolder, fileNameOnly+'.shx'))
    # take both paths and send to firescar dbf modification and regeneration
    fix_firescar_dbf(fireId, os.path.join(backupFolder, fileName), filePath)
    # TEST
    ####fix_firescar_dbf(fireId, filePath, '')
  else:
    print('ALREADY MODIFIED! '+filePath)


In [ ]:
# needs: IgnitionDate, ControlDate, Name,  lat, lon, Overlap IDS, from dfsm (dataframe super master) 
def fix_severity_dbf(fireId, inFilePath, outFilePath):
  #listFiles = filter(os.path.isfile, glob.glob(base_path +'Fire_CL-BI_BioBio/FireScar_CL-BI_BioBio_1985/'+'*.dbf') ) 
#for filePath in listFiles:  
#  print(filePath) 
#r = shapefile.Reader(os.path.join(output_path, 'testin', 'FireScar_CL-CO_ID20463_u700_19890120.dbf'))
#w = shapefile.Writer(os.path.join(output_path, 'testout', 'FireScar_CL-CO_ID20463_u700_19890120.dbf'), encoding="utf8")
  r = shapefile.Reader(inFilePath)
  w = shapefile.Writer(outFilePath)
  # EXTRA check field names consistency
  ### FINAL FIRESCAR: 
  ### FINAL 
  #####Columnas Severity (15): FireID, Region, Season, Name, IgnDate, CtrlDate, IgnLat, IgnLon, Severity, TotalArea, AreaUnchS, AreaLowS, AreaModS, AreaHighS, AreaCONAF, 
  ####---
  assert (r.fields[1][0]=='AreaTotal' and r.fields[4][0]=='Severity') or (r.fields[1][0]=='Control' and r.fields[4][0]=='Inicio')
  #### SEVERITY TIPO A
  ### ['AreaTotal', 'Temporada', 'AreaINC_ha', 'Severity', 'AreaUnchS', 'AreaLowS', 'AreaHighS', 'Control', 'AreaMedS', 'Inicio', 'Region', 'Nombre_Inc', 'KEY']
  if r.fields[1][0]=='AreaTotal' and r.fields[4][0]=='Severity':
    # RENAME, 40 to FireName, Character 10 to FireID
    ### 0[extra] 1['AreaTotal', 2'Temporada', 3'AreaINC_ha', 4'Severity', 5'AreaUnchS', 6'AreaLowS', 7'AreaHighS', 8'Control', 9'AreaMedS', 10'Inicio', 11'Region', 12'Nombre_Inc', 13'KEY']
    w.fields= [['TotalArea', 'N', r.fields[1][2], r.fields[1][3]], ['Season', 'N', 4, 0], ['AreaCONAF', 'N', r.fields[3][2], r.fields[3][3]], ['Severity', 'N', r.fields[4][2], r.fields[4][3]],
               ['AreaUnchS', 'N', r.fields[5][2], r.fields[5][3]], ['AreaLowS', 'N', r.fields[6][2], r.fields[6][3]], ['AreaHighS', 'N', r.fields[7][2], r.fields[7][3]],
               ['CtrlDate', 'C', 10, 0], ['AreaModS', 'N', r.fields[9][2], r.fields[9][3]], ['IgnDate', 'C', 10, 0], ['Region', 'C', 14, 0],
               ['FireName', 'C', len(dfsm['FireName_CONAF'][fireId])+1, 0], ['FireID', 'C', 10, 0]]
    ### 0['AreaTotal', 1'Temporada', 2'AreaINC_ha', 3'Severity', 4'AreaUnchS', 5'AreaLowS', 6'AreaHighS', 7'Control', 8'AreaMedS', 9'Inicio', 10'Region', 11'Nombre_Inc', 12'KEY']
    myorder = [12, 10, 1, 11, 9, 7, 3, 0, 4, 5, 8, 6, 2]

  # ['AreaHighS', 'N', 18, 15], ['Control', 'C', 8, 0], ['AreaMedS', 'N', 17, 15], ['Inicio', 'C', 14, 0], ['Region', 'C', 8, 0], ['NDVI_mean', 'N', 19, 15], ['Nombre_Inc', 'C', 15, 0], 
  # ['Area_Poly', 'N', 18, 15], ['KEY', 'N', 5, 0]]
  # w.fields = r.fields[1:] # skip first deletion field 

  #### SEVERITY TIPO B
  ###  ['Control', 'AreaTotal', 'AreaMedS', 'Inicio', 'Temporada', 'AreaINC_ha', 'Region', 'Severity', 'AreaUnchS', 'Nombre_Inc', 'AreaLowS', 'AreaHighS', 'KEY']  
  elif (r.fields[1][0]=='Control' and r.fields[4][0]=='Inicio'):
    w.fields= [['CtrlDate', 'C', 10, 0], ['TotalArea', 'N', r.fields[2][2], r.fields[2][3]], ['AreaModS', 'N', r.fields[3][2], r.fields[3][3]], ['IgnDate', 'C', 10, 0],
               ['Season', 'N', 4, 0], ['AreaCONAF', 'N', r.fields[6][2], r.fields[6][3]], ['Region', 'C', 14, 0], ['Severity', 'N', r.fields[8][2], r.fields[8][3]],
               ['AreaUnchS', 'N', r.fields[9][2], r.fields[9][3]], ['FireName', 'C', len(dfsm['FireName_CONAF'][fireId])+1, 0], ['AreaLowS', 'N', r.fields[11][2], r.fields[11][3]], 
               ['AreaHighS', 'N', r.fields[12][2], r.fields[12][3]], ['FireID', 'C', 10, 0]]
    # 0['Control', 1'AreaTotal', 2'AreaMedS', 3'Inicio', 4'Temporada', 5'AreaINC_ha', 6'Region', 7'Severity', 8'AreaUnchS', 9'Nombre_Inc', 10'AreaLowS', 11'AreaHighS', 12'KEY']  
    myorder = [12, 6, 4, 9, 3, 0, 7, 1, 8, 10, 2, 11, 5]    
    
  w.fields = [w.fields[i] for i in myorder]
  # ADD NEW FIELDS

  w.field('IgnLat', 'N', decimal=4)
  w.field('IgnLon', 'N', decimal=4)
  finalorder = [0, 1, 2, 3, 4, 5, 13, 14, 6, 7, 8, 9, 10, 11, 12] 
  w.fields = [w.fields[i] for i in finalorder]
  ####w.field('OverlapIDs', 'C', '40')

  # REORDER OK

  print(w.fields)

  for shaperec in r.iterShapeRecords():
    newline = shaperec.record  
    newline = [newline[i] for i in myorder]
    #####Columnas Severity (15): FireID, Region, Season, Name, IgnDate, CtrlDate, *** Severity, TotalArea, AreaUnchS, AreaLowS, AreaModS, AreaHighS, AreaCONAF, IgnLat, IgnLon,

    newline[0] = 'ID'+str(newline[0])
    newline[3] = dfsm['FireName_CONAF'][fireId]
    newline[4] =  dfsm['IgnitionDate_CONAF'][fireId]
    newline[5] =  dfsm['ControlDate_CONAF'][fireId]        
    newline.append(dfsm['Latitude [°]'][fireId]) # OBS: IS FLOAT
    newline.append(dfsm['Longitude [°]'][fireId])
    ##newline.append(dfsm['OverlapIDs'][fireId])
    newline = [newline[i] for i in finalorder]
    print (newline)
    w.record(*newline)
    w.shape(shaperec.shape)  
  w.close()
  # test
  #w2 = shapefile.Reader())
  #print (w2.fields)
  #print (w2.records)

In [ ]:
def check_severity_dbf(fireId, filePath, backupFolder):
  print('check severity dbf for ',filePath)
  print('with backup folder ',backupFolder)

  fileName=os.path.basename(filePath)
  dirName = os.path.dirname(filePath) 
  # first identify dbf, shp and shx and MOVE to backup with same structure (create if not existing)
  #print (fileName)
  (fileNameOnly, fileExt)=os.path.splitext(fileName) 
  f2 = os.path.join(dirName, fileNameOnly+'.shp')
  f3 = os.path.join(dirName, fileNameOnly+'.shx')
  assert (os.path.getsize(filePath) and os.path.getsize(f2) and os.path.getsize(f3))  
   #os.path.getsize(path) 
  print('OK ALL PATHS'+f3)
  # OPEN AND CHECK FIELDS!
  r = shapefile.Reader(filePath)
  print("R.FIELDS!")
  print(r.fields)
  #### SEVERITY TIPO A
  ### AreaTotal,N,17,15 Temporada,N,4,0 AreaINC_ha,N,4,2        Severity,N,1,0  AreaUnchS,N,1,0 AreaLowS,N,1,0  AreaHighS,N,17,15         ### Control,C,14    AreaMedS,N,1,0  Inicio,C,14     Region,C,8      Nombre_Inc,C,10 KEY,N,6,0  
  #### SEVERITY TIPO A
  ###  ['AreaTotal', 'Temporada', 'AreaINC_ha', 'Severity', 'AreaUnchS', 'AreaLowS', 'AreaHighS', 'Control', 'AreaMedS', 'Inicio', 'Region', 'Nombre_Inc', 'KEY']

  #### SEVERITY TIPO B
  ###  ['Control', 'AreaTotal', 'AreaMedS', 'Inicio', 'Temporada', 'AreaINC_ha', 'Region', 'Severity', 'AreaUnchS', 'Nombre_Inc', 'AreaLowS', 'AreaHighS', 'KEY']
  
  #### REF FIRESCAR
  #### ['NIR_min', 'N', 4, 0], ['AreaTotal', 'N', 17, 15], ['Temporada', 'N', 4, 0], ['AreaINC_ha', 'N', 4, 2], ['AreaUnchS', 'N', 1, 0], ['AreaLowS', 'N', 1, 0], 
  # ['AreaHighS', 'N', 18, 15], ['Control', 'C', 8, 0], ['AreaMedS', 'N', 17, 15], ['Inicio', 'C', 14, 0], ['Region', 'C', 8, 0], ['NDVI_mean', 'N', 19, 15], ['Nombre_Inc', 'C', 15, 0], 
  # ['Area_Poly', 'N', 18, 15], ['KEY', 'N', 5, 0]]

  #### All files have different definitions for the ranges .. only names are standard
  #if (r.fields==[('DeletionFlag', 'C', 1, 0),
  #      ['NIR_min', 'N', 3, 0], ['AreaTotal', 'N', 19, 15], ['Temporada', 'N', 4, 0], ['AreaINC_ha', 'N', 6, 4], ['AreaUnchS', 'N', 1, 0], 
  #              ['AreaLowS', 'N', 1, 0], ['AreaHighS', 'N', 19, 15], ['Control', 'C', 14, 0], ['AreaMedS', 'N', 1, 0], ['Inicio', 'C', 14, 0], ['Region', 'C', 8, 0], 
  #              ['NDVI_mean', 'N', 19, 15], ['Nombre_Inc', 'C', 11, 0], ['Area_Poly', 'N', 20, 15], ['KEY', 'N', 5, 0]]):

  if (r.fields[1][0]=='Control' and r.fields[4][0]=='Inicio') or (r.fields[1][0]=='AreaTotal' and r.fields[4][0]=='Severity'):
    # then send to fix function 
    # move original files to backup
    print ('SEV MOVE '+filePath +' to '+os.path.join(backupFolder, fileName))
    os.rename(filePath, os.path.join(backupFolder, fileName))
    print ('SEV MOVE '+f2 +' to '+os.path.join(backupFolder, fileNameOnly+'.shp'))
    os.rename(f2, os.path.join(backupFolder, fileNameOnly+'.shp'))
    print ('SEV MOVE '+f3 +' to '+os.path.join(backupFolder, fileNameOnly+'.shx'))
    os.rename(f3, os.path.join(backupFolder, fileNameOnly+'.shx'))
    # take both paths and send to firescar dbf modification and regeneration
    fix_severity_dbf(fireId, os.path.join(backupFolder, fileName), filePath)
    # TEST
    ####fix_firescar_dbf(fireId, filePath, '')
  else:
    print('ALREADY MODIFIED! '+filePath)

In [ ]:
def traverse_regions(folderFilter, subFolderFilter, yearFilter, idFilter):
  # traverse all regions
  # load needed files
  # SUPERMASTER!
  print('start')
  listFolders = filter(os.path.isdir, glob.glob(base_path + folderFilter + '*') )  
  for folderPath in listFolders:  
    print(folderPath)  
    # Filter regions - years that match filters
    listSubfolders = filter(os.path.isdir, glob.glob(folderPath + '/'+subFolderFilter + '*'+yearFilter) )  
    folderRegion=os.path.basename(folderPath)   
    if flag_fixDBF:
      backup_region_path= os.path.join(base_path, 'Report', 'backup', folderRegion)
      create_folder(backup_region_path)

    print(folderPath + '/'+folderFilter + '*')
    # traverse all region-season    
    for subfolderPath in listSubfolders:   
      listFiles = filter(os.path.isfile, glob.glob(subfolderPath + '/*') )
      folderSeason=os.path.basename(subfolderPath)            
      print(folderSeason)
      if flag_fixDBF:
        backup_season_path= os.path.join(base_path, 'Report', 'backup', folderRegion, folderSeason)        
        create_folder(backup_season_path)
      for filePath in listFiles:   
        # Ex: FireScar_CL-BI_ID151372_u390_20101211        
        fileName=os.path.basename(filePath)
        dirName = os.path.dirname(filePath) # for print only
        print(fileName)
        (fileNameOnly, fileExt)=os.path.splitext(fileName) 
        (fireType, fireRegion, fireId, fireThreshold, fireDate)=fileNameOnly.split("_")
        #if (idFilter != fireId):
        #  continue

        #(fireType, fireRegion, fireId, fireThreshold, fireDate)=fileNameOnly.split("_")
        if fileExt == '.xml':
          remove_xml_file(fileExt,filePath)
          continue
        #if fireType=='FireScar' and fileExt == '.dbf' and flag_fixDBF == True:
        #  check_firescar_dbf(fireId, filePath, backup_season_path)
        
        if fireType=='Severity' and fileExt == '.dbf' and flag_fixDBF == True:
          check_severity_dbf(fireId, filePath, backup_season_path)
        # fix_firescar_dbf(os.path.join(backupFolder, fileName), filePath, IgnDate, CtrlDate, FireName, IgnLat, IgnLon, OverlapIds)
        #if fireType=='Severity' and fileExt == '.shp' and flag_fixDBF == True:
        #  fix_severity_dbf(filePath, dirName, fireRegion, fireId, fireThreshold, fireDate, fileExt)
        #dfsm['ControlDate_CONAF']['ID8301']


In [ ]:
flag_summary = 0
flag_printTif = 0
flag_modifDate = 0
flag_details = 0
flag_fixDBF = 1
regionFilter=''
regionFilter = '_CL-'
folderFilter='Fire'+regionFilter
subFolderFilter = 'FireScar'+regionFilter
yearFilter = ''
#yearFilter = '2005'
#idFilter='ID38899'
idFilter=''
#idFilter='ID15228'



traverse_regions('Fire'+regionFilter, 'FireScar'+regionFilter, yearFilter, idFilter)


Streaming output truncated to the last 5000 lines.
ImgPosF_CL-LI_ID146602_u450_20091231.tif
ImgPreF_CL-LI_ID146602_u450_20091231.tif
ImgPreF_CL-LI_ID146716_u470_20091213.tif
RdNBR_CL-LI_ID146494_u350_20100111.tif
Severity_CL-LI_ID146494_u350_20100111.tif
FireScar_CL-LI_ID146494_u350_20100111.tif
ImgPosF_CL-LI_ID146494_u350_20100111.tif
FireScar_CL-LI_ID146494_u350_20100111.fix
FireScar_CL-LI_ID146494_u350_20100111.prj
Severity_CL-LI_ID146494_u350_20100111.shx
Severity_CL-LI_ID146494_u350_20100111.cpg
Severity_CL-LI_ID146494_u350_20100111.prj
FireScar_CL-LI_ID146494_u350_20100111.cpg
ImgPreF_CL-LI_ID146494_u350_20100111.tif
Severity_CL-LI_ID146494_u350_20100111.shp
Severity_CL-LI_ID146494_u350_20100111.fix
Severity_CL-LI_ID146494_u350_20100111.dbf
check severity dbf for  drive/Shared drives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Fire_CL-LI_Ohiggins/FireScar_CL-LI_Ohiggins_2010/Severity_CL-LI_ID146494_u350_20100111.dbf
with back